<a href="https://www.kaggle.com/code/tunguz/tps-01-21-linear-xgboost?scriptVersionId=109963579" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
%matplotlib inline

At this point the Kaaggle Docker environment does not support XGBoost 1.3+, so we'll have to install it manually.

In [2]:
import xgboost as xgb
xgb.__version__


'1.2.1'

In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import gc


In [4]:


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tabular-playground-series-jan-2021/sample_submission.csv
/kaggle/input/tabular-playground-series-jan-2021/train.csv
/kaggle/input/tabular-playground-series-jan-2021/test.csv


In [5]:
train = pd.read_csv('../input/tabular-playground-series-jan-2021/train.csv')
test  = pd.read_csv('../input/tabular-playground-series-jan-2021/test.csv')
sub = pd.read_csv('../input/tabular-playground-series-jan-2021/sample_submission.csv')

In [6]:
test.shape

(200000, 15)

In [7]:
columns = test.columns[1:]
target = train['target'].values

In [8]:
train_oof = np.zeros((300000,))
test_preds = 0
train_oof.shape

(300000,)

In [9]:
## Best hyperparameters from the following notebook: https://www.kaggle.com/hamzaghanmi/xgboost-hyperparameter-tuning-using-optuna

Best_trial= {'booster': 'gblinear', 
             'learning_rate': 0.1,
             'random_state': 2020,
            }

In [10]:
test = xgb.DMatrix(test[columns])

In [11]:
NUM_FOLDS = 10
kf = KFold(n_splits=NUM_FOLDS, shuffle=True, random_state=0)

for f, (train_ind, val_ind) in tqdm(enumerate(kf.split(train, target))):
        #print(f'Fold {f}')
        train_df, val_df = train.iloc[train_ind][columns], train.iloc[val_ind][columns]
        train_target, val_target = target[train_ind], target[val_ind]
        
        train_df = xgb.DMatrix(train_df, label=train_target)
        val_df = xgb.DMatrix(val_df, label=val_target)
        
        model =  xgb.train(Best_trial, train_df, 5000)
        temp_oof = model.predict(val_df)
        temp_test = model.predict(test)

        train_oof[val_ind] = temp_oof
        test_preds += temp_test/NUM_FOLDS
        
        print(mean_squared_error(temp_oof, val_target, squared=False))

1it [02:07, 127.52s/it]

0.7247509309138365


2it [04:15, 127.74s/it]

0.7263071071005636


3it [06:22, 127.53s/it]

...

0.7268919538049368


9it [19:07, 127.46s/it]

0.7277903130211663


10it [21:16, 127.67s/it]

0.7230492139649227


In [12]:
0.6959799893005467


0.6959799893005467

In [13]:
mean_squared_error(train_oof, target, squared=False)


0.7262345812325547

In [14]:
np.save('train_oof', train_oof)
np.save('test_preds', test_preds)

In [15]:
sub['target'] = test_preds
sub.to_csv('submission.csv', index=False)

In [16]:
sub.head()

,id,target
0,0,8.053506
1,2,7.597546
2,6,7.950829
3,7,7.941919
4,10,8.060814
